<a href="https://colab.research.google.com/github/vinnik-dmitry07/Chatbot/blob/main/chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install --quiet parlai

     |████████████████████████████████| 1.4MB 5.7MB/s 
     |████████████████████████████████| 3.2MB 37.7MB/s 
     |████████████████████████████████| 204kB 34.4MB/s 
     |████████████████████████████████| 317kB 54.6MB/s 
     |████████████████████████████████| 61kB 9.5MB/s 
     |████████████████████████████████| 7.0MB 22.9MB/s 
     |████████████████████████████████| 7.2MB 59.5MB/s 
     |████████████████████████████████| 163kB 64.0MB/s 
     |████████████████████████████████| 51kB 8.5MB/s 
     |████████████████████████████████| 81kB 11.6MB/s 
     |████████████████████████████████| 245kB 58.9MB/s 
     |████████████████████████████████| 215kB 54.4MB/s 
     |████████████████████████████████| 51kB 8.5MB/s 
     |████████████████████████████████| 133kB 58.0MB/s 
     |████████████████████████████████| 133kB 54.7MB/s 
     |████████████████████████████████| 2.8MB 58.2MB/s 
     |████████████████████████████████| 40kB 217kB/s 
     |████████████████████████████████| 552kB 57.0MB/s 
  

In [3]:
!nvidia-smi

Fri Feb 12 14:22:53 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.39       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [37]:
import json

with open('/content/drive/MyDrive/result.json', 'r', encoding='utf8') as f:
    raw_messages = json.load(f)['messages']

In [38]:
from datetime import datetime

filtered_messages = []
for msg in raw_messages:
    if (
            'from' in msg and
            'from_id' in msg and
            'mime_type' not in msg and
            msg['text'] and
            isinstance(msg['text'], str)
    ):
        msg['date'] = datetime.strptime(msg['date'], '%Y-%m-%dT%H:%M:%S')
        filtered_messages.append(msg)

In [39]:
from datetime import timedelta

episode_dt = timedelta(minutes=5)
joined_messages = [filtered_messages[0]]
for i in range(1, len(filtered_messages)):
    if (
            filtered_messages[i - 1]['from_id'] == filtered_messages[i]['from_id'] and
            filtered_messages[i - 1]['date'] - filtered_messages[i]['date'] <= episode_dt
    ):
        joined_messages[-1]['text'] += ' ' + filtered_messages[i]['text']
    else:
        joined_messages.append(filtered_messages[i])

In [40]:
def partition(alist, indices):
    return [alist[a:b] for a, b in zip([0] + indices, indices + [None])]

time_diffs = [joined_messages[i + 1]['date'] - joined_messages[i]['date'] for i in range(len(joined_messages) - 1)]
split_positions = [i + 1 for i in range(len(time_diffs)) if time_diffs[i] > episode_dt]
episodes = partition(joined_messages, split_positions)
assert len(split_positions) >= 3, 'Not enough episodes, try decrease "episode_dt"'

In [41]:
def save_jsonl(episodes_part, suffix, human_readable=False):
    with open(f'data_{suffix}.jsonl', 'w', **({'encoding': 'utf8'} if human_readable else {})) as outfile:
        for episode in episodes_part:
            dialog = [
                {
                    'id': i % 2,
                    'text': msg['text'].replace('\n', ' '),
                } for i, msg in enumerate(episode)
            ]

            episode = {'dialog': [dialog]}
            json.dump(episode, outfile, **({'ensure_ascii': False} if human_readable else {}))
            outfile.write('\n')

In [42]:
import numpy as np

train_part, test_part, valid_part = 0.9, 0.05, 0.05
assert train_part + test_part + valid_part == 1

train, test, valid = np.split(episodes, [
    int(train_part * len(episodes)),
    int((train_part + test_part) * len(episodes)),
])

save_jsonl(train, 'train')
save_jsonl(test, 'test')
save_jsonl(valid, 'valid')


/usr/local/lib/python3.6/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [ ]:
!rm -rf from_pretrained
!mkdir -p from_pretrained

from parlai.scripts.train_model import TrainModel

TrainModel.main(
    task='jsonfile',
    jsonfile_datapath='data',
    jsonfile_datatype_extension=True,

    model='transformer/generator',
    model_file='from_pretrained/model',
    
    init_model='zoo:tutorial_transformer_generator/model',

    n_heads=16, n_layers=8, n_positions=512, text_truncate=512,
    label_truncate=128, ffn_size=2048, embedding_size=512,
    activation='gelu', variant='xlm',
    dict_lower=True, dict_tokenizer='bpe',
    dict_file='zoo:tutorial_transformer_generator/model.dict',
    learn_positional_embeddings=True,
    
    lr=1e-5, optimizer='adam',
    warmup_updates=100,
    validation_metric='ppl',
    validation_every_n_secs=60 * 60,
    save_every_n_secs=10 * 60,
    
    batchsize=12, fp16=True, fp16_impl='mem_efficient',
    
    skip_generation=True,
    
    dynamic_batching='full',

    label_turns='both',
)

14:39:11 | building dictionary first...
14:39:11 | No model with opt yet at: from_pretrained/model(.opt)
14:39:11 | your model is being loaded with opts that do not exist in the model you are initializing the weights with: allow_missing_init_opts: False,download_path: None,loglevel: info,dynamic_batching: full,datapath: /usr/local/lib/python3.6/dist-packages/data,tensorboard_logdir: None,jsonfile_datapath: data,jsonfile_datatype_extension: True,label_turns: both,n_encoder_layers: -1,n_decoder_layers: -1,model_parallel: False,beam_block_full_context: True,beam_length_penalty: 0.65,topk: 10,topp: 0.9,beam_delay: 30,beam_block_list_filename: None,temperature: 1.0,compute_tokenized_bleu: False,interactive_mode: False,fp16_impl: mem_efficient,force_fp16_tokens: False,adafactor_eps: (1e-30, 0.001),history_reversed: False,history_add_global_end_token: None,special_tok_lst: None,bpe_vocab: None,bpe_merge: None,bpe_add_prefix_space: None,hf_skip_special_tokens: True,max_lr_steps: -1,invsqrt_lr_

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:216: UserWarning: Please also save or load the state of the optimizer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


14:49:33 | time:606s total_exs:44176 epochs:0.07
    clip  ctpb  ctps  exps  exs  fp16_loss_scalar  gnorm  gpu_mem  loss    lr  ltpb  ltps   ppl  token_acc  \
   .9375  6285  8317 22.49  272             16384  17.96    .4628 3.032 1e-05 138.2 182.9 20.73      .3879   
    total_train_updates  tpb  tps   ups  
                   1376 6423 8500 1.323

14:49:39 | Overflow: setting loss scale to 16384.0
14:49:40 | Overflow: setting loss scale to 16384.0
14:49:43 | time:616s total_exs:44540 epochs:0.07
    clip  ctpb  ctps  exps  exs  fp16_loss_scalar  gnorm  gpu_mem  loss    lr  ltpb  ltps   ppl  token_acc  \
   .9130  6420 14539 35.84  364             16384  17.99    .4628 3.341 1e-05 104.5 236.7 28.24      .3669   
    total_train_updates  tpb   tps   ups  
                   1399 6524 14776 2.265

14:49:47 | Overflow: setting loss scale to 16384.0
14:49:49 | Overflow: setting loss scale to 16384.0
14:49:53 | time:626s total_exs:44936 epochs:0.07
    clip  ctpb  ctps  exps  exs  fp16_los